In [3]:
import numpy as np
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score 
import dlib
import torch
from torch import nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader

from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# 1. 加载 LFW 数据集
# 只选择至少有 70 张图像的人员，减少类别数量，加快特征提取
lfw_dataset = fetch_lfw_people(min_faces_per_person=70, resize=0.5)

X = lfw_dataset.images  # 图像数据，形状为 (n_samples, h, w)
y = lfw_dataset.target   # 标签，形状为 (n_samples,)
target_names = lfw_dataset.target_names  # 名字列表
n_classes = target_names.shape[0]        # 类别数量

print(f"样本数量: {X.shape[0]}")
print(f"类别数量: {n_classes}")

# 2. 标签编码
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 3. 数据集划分
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.25, random_state=42, stratify=y_encoded
)

# 4. 数据预处理和自定义数据集
class LFWImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images  # NumPy 数组
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        
        # 将灰度图像转换为 RGB
        img = np.stack((img,)*3, axis=-1)
        img = Image.fromarray(np.uint8(img))
        
        if self.transform:
            img = self.transform(img)
        
        return img, label

# 定义图像预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图像尺寸
    transforms.ToTensor(),          # 转换为 Tensor，像素值归一化到 [0,1]
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet 数据集的均值
        std=[0.229, 0.224, 0.225]    # ImageNet 数据集的标准差
    ),
])

# 创建数据集和数据加载器
batch_size = 64

train_dataset = LFWImageDataset(X_train, y_train, transform=transform)
test_dataset = LFWImageDataset(X_test, y_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 5. 特征提取
# 加载预训练的 ResNet 模型，并移除最后的全连接层
# resnet = models.resnet18(pretrained=True)
# resent模型
resnet = dlib.face_recognition_model_v1('./dlib_face_recognition_resnet_model_v1.dat')
feature_extractor = nn.Sequential(*list(resnet.children())[:-1])
feature_extractor.eval()

# 将模型移动到设备（CPU 或 GPU）
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
feature_extractor = feature_extractor.to(device)

# 定义特征提取函数
def extract_features(dataloader):
    features = []
    labels = []
    with torch.no_grad():
        for imgs, lbls in tqdm(dataloader):
            imgs = imgs.to(device)
            output = feature_extractor(imgs)
            output = output.view(output.size(0), -1)  # 展平成一维向量
            features.append(output.cpu().numpy())
            labels.extend(lbls.numpy())
    features = np.vstack(features)
    labels = np.array(labels)
    return features, labels

# 提取训练集和测试集的特征
print("提取训练集特征...")
train_features, train_labels = extract_features(train_loader)
print("提取测试集特征...")
test_features, test_labels = extract_features(test_loader)

# 6. 分类器训练
from sklearn.svm import SVC

# 使用线性核的 SVM 分类器
clf = SVC(kernel='linear', probability=True)
clf.fit(train_features, train_labels)

# 7. 模型评估
y_pred = clf.predict(test_features)

# 输出分类报告
print("分类报告:")
print(classification_report(test_labels, y_pred, target_names=target_names))

# 绘制混淆矩阵
cm = confusion_matrix(test_labels, y_pred)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=target_names, yticklabels=target_names)
plt.xlabel('预测类别')
plt.ylabel('真实类别')
plt.title('混淆矩阵')
plt.show()

accuracy_score(y_test, y_pred)

样本数量: 1288
类别数量: 7


RuntimeError: Unable to open ./weights/dlib_face_recognition_resnet_model_v1.dat for reading.